In [66]:
# Author : Sahil Chitnis
# Dataset : https://drive.google.com/drive/folders/1s1-174qlu_ekiKcGXeutP5tvrVdjmsQF


In [ ]:
# Install some libraries/packages required for Google Collab

#!pip install noisereduce
#!pip install torchaudio
#!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
#!pip install pyaudio

In [142]:
# Step0: Import all libraries required for this project

import librosa
import os
import numpy as np
import noisereduce as nr
import torchaudio
import tensorflow
import numpy
import torch
import random
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,MaxPooling1D,Conv1D
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,TimeDistributed
from keras.optimizers import Adam,SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
import random
from keras import optimizers
import datetime
import matplotlib.pyplot as plt
import noisereduce as nr
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import IPython
import os
import pyaudio

In [143]:
#!unzip ./Dataset_audio.zip

In [144]:
# Step1 : Data Cleaning and Feature extraction step

# Step1.a) Data augmentation - Shift the signal to left/right by some % 
#          This step is needed to increase samples
def time_shift(aud, shift_limit):
  sig,sr = aud
  _, sig_len = sig.shape
  shift_amt = int(random.random() * shift_limit * sig_len)
  return (sig.roll(shift_amt), sr)

# Step1.b) Transform data and extract features :
#
# a) Data Augmentation
# b) Noise reduction
# c) Data Trimming
# d) Perform Short-time Fourier transform to extract features
# e) Save features for future use

def compute_and_save_transform(file, name, activity, subject):

    # read the audio data
    audio_data, sample_rate = librosa.load(file)
    #aud = torchaudio.load(file)
    #data = []

    # Performing data augmentation
    #aud1 = time_shift(aud, 1)
    #for elem in aud1[0][0]:
    #  data.append(elem)
    #audio_data = np.array(data)
    
    

    # Perform noise reduction
    noise_audio = audio_data[0:35000]  
    noise_removed_audio = nr.reduce_noise(audio_clip=audio_data, noise_clip=noise_audio, verbose=False)
    
    
    
    # Trim the silence in the data by setting silence threshold to 20DB, 
    # 512 samples per frame and 128 number of samples between analysis frames
    trimmed_audio, index = librosa.effects.trim(noise_removed_audio, top_db=20, frame_length=512, hop_length=128)
    
    
    
    # Perform Short-time Fourier transform  to extract features
    stft_feat = np.abs(librosa.stft(trimmed_audio, n_fft=512, hop_length=256, win_length=512))
    

    # Save the features to folder "Transformed_Audio_Features"
    np.save("Transformed_Audio_Features/" + subject + "_" + name[:-4] + "_" + activity + ".npy", stft_feat)


In [ ]:
# Step1.c) Perform step1.b for all activities in all samples

activities = ['Writing', 'StandUp', 'Calling', 'Drinking', 'Clapping', 'Eating',
              'Exiting', 'Falling', 'OpeningPillContainer', 'LyingDown',
              'Reading', 'Sitting', 'SitStill', 'Sleeping', 'PickingObject',
              'Sweeping', 'UsingPhone', 'UseLaptop', 'WakeUp', 'Walking',
              'WashingHand', 'WatchingTV', 'WaterPouring', 'Entering']
    
samples = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09',
            's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17']

for activity in activities:
    for sample in samples:
        innerDir = sample + "/" + activity
        for file in os.listdir("./Dataset_audio/" + innerDir):
            if(file.endswith(".wav")):
                compute_and_save_transform("Dataset_audio/" + innerDir + "/" + file, file, activity, sample)
                print("Sample",sample, "Performing", activity, "in", file)

In [146]:
# Step2) Split Activities1,2,3 of each sample as per 
#        sample in 3 buckets into Training, Test, Validation based on 

#  Taking 12 Training samples
train_samples = ['s07', 's16', 's09', 's13', 's04', 's11', 's15', 's01', 's12', 's10', 's06', 's08']

# Taking 2 validation samples
validation_samples = ['s02', 's03']


# Taking 4 test samples
test_samples = ['s05', 's17']


def Split_data(path,sampleSize):
    
    
    Activities1 = ['Drinking', 'Eating', 'LyingDown', 'OpeningPillContainer', 
                          'PickingObject', 'Reading', 'SitStill', 'Sitting', 'Sleeping', 
                          'StandUp', 'UseLaptop', 'UsingPhone', 'WakeUp', 'Walking', 
                          'WaterPouring', 'Writing']
    
    Activities2 = ['Calling', 'Clapping', 'Falling', 'Sweeping', 'WashingHand', 'WatchingTV']
    
    Activities3 = ['Entering', 'Exiting']
    
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    X_validation = []
    Y_validation = []
    
    # Split data into Training, Validation, Test
    for file in os.listdir(path):
        if int(file.split("__")[1].split("_")[0])!=1:
          a = (np.load(path + file)).T
          activityLabel = file.split('_')[-1].split(".")[0]
          # Split data from Activities2
          if(activityLabel in Activities2):
                if file.split("_")[0] in train_samples:
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(activityLabel)
                elif file.split("_")[0] in validation_samples:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(activityLabel)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(activityLabel)
          # Split data from Activities3
          elif(activityLabel in Activities3):
                activityLabel = "Activities3"
                if file.split("_")[0] in train_samples:
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(activityLabel)
                elif file.split("_")[0] in validation_samples:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(activityLabel)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(activityLabel)
          # Split from remaining activities
          else:
                activityLabel = "other"
                if file.split("_")[0] in train_samples:
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(activityLabel)
                elif file.split("_")[0] in validation_samples:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(activityLabel)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(activityLabel)
    
    # Change lists to numpy arrays              
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    X_validation = np.array(X_validation)
    Y_validation = np.array(Y_validation)
    
    return X_train,Y_train,X_validation,Y_validation,X_test,Y_test      


In [147]:
# Step3) Functions to print output/data

# Re-sample the data
def reSample(data, samples):
    r = len(data)/samples #re-sampling ratio
    newdata = []
    for i in range(0,samples):
        newdata.append(data[int(i*r)])
    return np.array(newdata)

# Print ConfMatrix
def print_activity_data1(confMatrix):
        s = "ACTIVITY Confusion Matrix:\n"
        for i in range(len(confMatrix)):
            s += lb.inverse_transform([i])[0] + "\t|"
        print(s[:-1])
        for i in range(len(confMatrix)):
            s = ""
            for j in range(len(confMatrix)):
                s += str(confMatrix[i][j])
                s += "\t|"
            print(lb.inverse_transform([i])[0],"\t|", s[:-1])
        print()
        
def print_activity_data2(confMatrix):
        s = "ACTIVITY  Matrix:\n"
        for i in range(len(confMatrix)):
            s += lb.inverse_transform([i])[0] + "\t|"
        print(s[:-1])
        for i in range(len(confMatrix)):
            s = ""
            for j in range(len(confMatrix)):
                val = confMatrix[i][j]/float(sum(confMatrix[i]))
                s += str(round(val,2))
                s += "\t|"
            print(lb.inverse_transform([i])[0],"\t|", s[:-1])
        print()


# Display the result
def DisplayResult():
  predictions = [np.argmax(y) for y in result]
  expected = [np.argmax(y) for y in y_test]

  confMatrix = []
  num_labels=y_test[0].shape[0]
  for i in range(num_labels):
      r = []
      for j in range(num_labels):
          r.append(0)
      confMatrix.append(r)

  n_tests = len(predictions)
  for i in range(n_tests):        
      confMatrix[expected[i]][predictions[i]] += 1

  # Print activity data 
  print_activity_data1(confMatrix)
  print_activity_data2(confMatrix)

In [148]:
featuresPath = "Transformed_Audio_Features/"

# Split data into Train, Validation, Test
X_train,Y_train,X_validation,Y_validation,X_test,Y_test  = Split_data(featuresPath,250)

In [149]:
n_samples = len(Y_train)
print("Number of training samples: " + str(n_samples))
order = np.array(range(n_samples))

# Shuffle Training data 
np.random.shuffle(order)
X_train = X_train[order]
Y_train = Y_train[order]


# Step4) Encode the labels for Y_train, Y_test and Y_validation

# Encode the labels
lb = LabelEncoder()

# Fit label encoder on Y_train, Y_test and Y_validation
y_train = np_utils.to_categorical(lb.fit_transform(Y_train))
y_test = np_utils.to_categorical(lb.fit_transform(Y_test))
y_validation = np_utils.to_categorical(lb.fit_transform(Y_validation))
num_labels = y_train.shape[1]

Number of training samples: 880


In [161]:

# Step5.1) Build Model:
#        a)  1st layer : Dense with o/p = 256 neurons, i/p = 257and RELU activation
#        b)  2nd layer : Dense with o/p = 256 neurons and RELU activation
#        c)  3rd layer : Dense with o/p = 128 neurons and RELU activation
#        d)  4th layer : Dense with o/p = 128 neurons, RELU activation and Dropout = 0.5
#        d)  5th layer : Dense with o/p = 128 neurons, RELU activation and Dropout = 0.5
#        e)  6th layer : Dense with o/p = 8 neurons (ie = num_labels), Softmax activation

num_labels = y_train.shape[1]
filter_size = 20

# Build Sequential model
model = Sequential()

#Layer1
model.add(Dense(256, input_shape=(257,)))
model.add(Activation('relu'))

#Layer2
model.add(Dense(256))
model.add(Activation('relu'))

#Layer3
model.add(Dense(128))
model.add(Activation('relu'))

#Layer4
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#Layer5
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#Layer6
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()

# Step5.2) Compile Model:
#          a) Loss is categorical_crossentropy
#          b) Optimizer is Adam optimizer

#lr_rates = [2e-3, 3e-1, 3e-2, 3e-3]


# Step5.3) Fit the Model:
#          a) Batch size is 10
#          b) Epoch's is 100
lr_rate = 3e-3
opt = keras.optimizers.Adam(learning_rate=lr_rate)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
model.fit(X_train, y_train, batch_size=10, epochs=100, validation_data=(X_validation,y_validation))
#    result = model.predict(X_test)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_137 (Dense)            (None, 256)               66048     
_________________________________________________________________
activation_137 (Activation)  (None, 256)               0         
_________________________________________________________________
dense_138 (Dense)            (None, 256)               65792     
_________________________________________________________________
activation_138 (Activation)  (None, 256)               0         
_________________________________________________________________
dense_139 (Dense)            (None, 128)               32896     
_________________________________________________________________
activation_139 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_140 (Dense)            (None, 128)             

Epoch 42/100
880/880 [==============================] - 1s 989us/step - loss: 0.4027 - accuracy: 0.8852 - val_loss: 0.4162 - val_accuracy: 0.8904
Epoch 43/100
880/880 [==============================] - 1s 937us/step - loss: 0.4272 - accuracy: 0.8795 - val_loss: 0.4390 - val_accuracy: 0.8630
Epoch 44/100
880/880 [==============================] - 1s 930us/step - loss: 0.4030 - accuracy: 0.8886 - val_loss: 0.4076 - val_accuracy: 0.8836
Epoch 45/100
880/880 [==============================] - 1s 970us/step - loss: 0.3946 - accuracy: 0.8909 - val_loss: 0.4038 - val_accuracy: 0.8904
Epoch 46/100
880/880 [==============================] - 1s 914us/step - loss: 0.3782 - accuracy: 0.8955 - val_loss: 0.4328 - val_accuracy: 0.8767
Epoch 47/100
880/880 [==============================] - 1s 989us/step - loss: 0.3826 - accuracy: 0.8875 - val_loss: 0.4349 - val_accuracy: 0.8699
Epoch 48/100
880/880 [==============================] - 1s 879us/step - loss: 0.3940 - accuracy: 0.8886 - val_loss: 0.3450 -

In [162]:
# Step5.4) Predict on test data using above Model
result = model.predict(X_test)

# Step5.5) Calculate accuracy
cnt = 0
for i in range(len(Y_test)):
    if(np.amax(result[i]) < 0.5):
      pred = np.argmax(result[i])
    else:
      pred = np.argmax(result[i])
    if np.argmax(y_test[i]) == pred:
        cnt+=1

acc = str(round( cnt*100 / float(len(Y_test)),2))
print("Accuracy: " + acc + "%")

# Step5.6) Display accuracy and ConfMatrix
DisplayResult()

Accuracy: 91.28%
ACTIVITY Confusion Matrix:
Activities3	|Calling	|Clapping	|Falling	|Sweeping	|WashingHand	|WatchingTV	|other	
Activities3 	| 14	|0	|0	|0	|0	|0	|0	|0	
Calling 	| 0	|6	|0	|2	|0	|0	|0	|4	
Clapping 	| 0	|0	|12	|0	|0	|0	|0	|0	
Falling 	| 0	|0	|0	|11	|1	|0	|0	|0	
Sweeping 	| 0	|0	|0	|0	|6	|0	|0	|0	
WashingHand 	| 0	|0	|0	|0	|0	|4	|0	|2	
WatchingTV 	| 0	|0	|0	|0	|1	|0	|1	|4	
other 	| 0	|0	|0	|2	|3	|0	|0	|145	

ACTIVITY  Matrix:
Activities3	|Calling	|Clapping	|Falling	|Sweeping	|WashingHand	|WatchingTV	|other	
Activities3 	| 1.0	|0.0	|0.0	|0.0	|0.0	|0.0	|0.0	|0.0	
Calling 	| 0.0	|0.5	|0.0	|0.17	|0.0	|0.0	|0.0	|0.33	
Clapping 	| 0.0	|0.0	|1.0	|0.0	|0.0	|0.0	|0.0	|0.0	
Falling 	| 0.0	|0.0	|0.0	|0.92	|0.08	|0.0	|0.0	|0.0	
Sweeping 	| 0.0	|0.0	|0.0	|0.0	|1.0	|0.0	|0.0	|0.0	
WashingHand 	| 0.0	|0.0	|0.0	|0.0	|0.0	|0.67	|0.0	|0.33	
WatchingTV 	| 0.0	|0.0	|0.0	|0.0	|0.17	|0.0	|0.17	|0.67	
other 	| 0.0	|0.0	|0.0	|0.01	|0.02	|0.0	|0.0	|0.97	



In [163]:
# Step6) Save Model for future use

model_path = r"Audio_Classification_Model/"
model_name = "audio_CNN_model"



model_json = model.to_json()

with open(model_path + model_name +".json", "w") as json_file:
    json_file.write(model_json)
    print("SAVED MODEL !!! Model saved to folder ./" + model_path + " as " + model_name + ".json")

# serialize weights to HDF5
model.save_weights(model_path + model_name + ".h5")
print("SAVED WEIGHTS !!! Serialized weights stored to folder ./" + model_path + " as " +model_name+ ".h5")

SAVED MODEL !!! Model saved to folder ./Audio_Classification_Model/ as audio_CNN_model.json
SAVED WEIGHTS !!! Serialized weights stored to folder ./Audio_Classification_Model/ as audio_CNN_model.h5
